In [12]:
import random
import os

import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import gurobipy as gp
from tqdm import trange, tqdm

In [13]:
def greedy_qap(F, D):    
    num_facilities = F.shape[0]
    remaining_facilities = set(range(num_facilities))
    remaining_locations = set(range(num_facilities))
    perm = [-1 for _ in range(num_facilities)]
    
    for i in range(num_facilities):
        best_cost = float('inf')
        best_facility, best_location = -1, -1
        
        for f in remaining_facilities:
            for l in remaining_locations:
                current_cost = sum([F[f][j] * D[l][perm[j]] for j in range(i) if perm[j] != -1])
                
                if current_cost < best_cost:
                    best_cost = current_cost
                    best_facility, best_location = f, l
                    
        perm[best_location] = best_facility
        remaining_facilities.remove(best_facility)
        remaining_locations.remove(best_location)
    
    # Compute final cost
    cost = 0
    for i in range(num_facilities):
        for j in range(num_facilities):
            cost += F[perm[i]][perm[j]] * D[i][j]
    
    return perm


In [14]:
def objective_function(permutation, distance, flow):
    n = len(permutation)
    total = 0
    for i in range(n):
        for j in range(n):
            total += flow[i][j] * distance[permutation[i]][permutation[j]]
    return total

def generate_neighbors(solution):
    neighbors = []
    for i in range(len(solution)):
        for j in range(i+1, len(solution)):
            neighbor = solution.copy()
            neighbor[i], neighbor[j] = solution[j], solution[i]
            neighbors.append(neighbor)
    return neighbors

def tabu_search(distance, flow, iters=1000, tabu_size=100):
    n = len(flow)

    best_solution = greedy_qap(distance, flow)
    best_cost = objective_function(best_solution, distance, flow)
    
    current_solution = best_solution
    current_cost = best_cost
    
    tabu_list = []

    for _ in range(iters):
        neighbors = generate_neighbors(current_solution)
        best_neighbor = None
        best_neighbor_cost = float('inf')
        
        for neighbor in neighbors:
            if neighbor not in tabu_list:
                cost = objective_function(neighbor, distance, flow)
                if cost < best_neighbor_cost:
                    best_neighbor_cost = cost
                    best_neighbor = neighbor
        
        if best_neighbor:
            tabu_list.append(best_neighbor)
            if len(tabu_list) > tabu_size:
                tabu_list.pop(0)
            
            current_solution = best_neighbor
            current_cost = best_neighbor_cost
            
            if current_cost < best_cost:
                best_cost = current_cost
                best_solution = current_solution

    return best_solution, best_cost

In [15]:
def visualize_qap_solution(distance_matrix, flow_matrix, solution):
    num_facilities = distance_matrix.shape[0]
    cost_matrix = np.zeros((num_facilities, num_facilities))

    for i in range(num_facilities):
        for j in range(num_facilities):
            cost_matrix[i][j] = distance_matrix[i][j] * flow_matrix[solution[i]][solution[j]]

    plt.matshow(cost_matrix, cmap='viridis')
    plt.colorbar(label='Cost')
    plt.title('Cost for each Facility-Location Mapping')
    plt.xlabel('Facilities')
    plt.ylabel('Locations')
    plt.show()

In [16]:
# set up params for plots
SMALL, MED, LARGE, LW = 18, 24, 30, 3
plt.rc('axes', titlesize=SMALL)    # fontsize of the axes title
plt.rc('axes', labelsize=SMALL)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL) # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL) # fontsize of the tick labels
plt.rc('legend', fontsize=MED)   # legend fontsize
plt.rc('font', size=SMALL)       # controls default text sizes

In [17]:
# for managing file opening and closing
def read_integers(filename):
    with open(filename) as f:
        return [int(elem) for elem in f.read().split()]

# Using raw string for the path
file_path = '../QAPInstances/tai30b.dat'  
file_it = iter(read_integers(file_path))

# Number of points
n = next(file_it)

# Distance between locations
w = np.array([[next(file_it) for j in range(n)] for i in range(n)])
d = np.array([[next(file_it) for j in range(n)] for i in range(n)])

solution,cost = tabu_search(w, d)
visualize_qap_solution(w, d, solution)
print(solution)
print(cost)

KeyboardInterrupt: 